<a href="https://colab.research.google.com/github/farsim-hossain/python-projects/blob/main/Hypertuning%20With%20Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hyperparameter Tuning With Keras 

We will use Keras tuner to perform hypertuning.

In [1]:
import tensorflow as tf 
from tensorflow import keras 

In [4]:
pip install -q -U keras-tuner

     |████████████████████████████████| 71kB 6.4MB/s 


In [5]:
import kerastuner as kt 


## Loading the dataset 

We will use the fashion mnist dataset 

In [6]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


## Normalizing 

In [7]:
img_train = img_train.astype('float32')/255.0
img_test = img_test.astype('float32')/255.0

## Defining the model 

When you build a model for hypertuning, you also define the hyperparameter search space in addition to the model architecture. The model you set up for hypertuning is called a hypermodel.

You can define a hypermodel through two approaches:

By using a model builder function
By subclassing the HyperModel class of the Keras Tuner API

In this tutorial, you use a model builder function to define the image classification model. The model builder function returns a compiled model and uses hyperparameters you define inline to hypertune the model.

In [13]:
def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28, 28)))

  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
  model.add(keras.layers.Dense(units=hp_units, activation='relu'))
  model.add(keras.layers.Dense(10))

  # Tune the learning rate for the optimizer
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  return model

## Instantiate the tuner and perform hyper tuning

We will use Hyperband tuner. To instantiate the Hyperband tuner, you must specify the hypermodel, the objective to optimize and the maximum number of epochs to train (max_epochs).

In [14]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt')

## Early Stopping 

hyperband algorithm uses adaptive resources allocation and early stopping to quickly converge on a high performing model.

In [15]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience =5)

Now we have the hyperband ready which takes the model in its arguements, we have the early stopping ready. Now lets search the hyperparameters.

**The Search method is the method that will RUN the search of the tuner object**

In [16]:
tuner.search(img_train, label_train, epochs = 50, validation_split = 0.2, callbacks = [stop_early])

# get the optimal hyper parameters 

best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

Trial 30 Complete [00h 00m 26s]
val_accuracy: 0.8559166789054871

Best val_accuracy So Far: 0.8877500295639038
Total elapsed time: 00h 07m 31s
INFO:tensorflow:Oracle triggered exit

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 384 and the optimal learning rate for the optimizer
is 0.001.



## Train The model 

Find the optimal number of epochs to train the model with the hyperparameters obtained from the search.

In [18]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs 

model = tuner.hypermodel.build(best_hps)
# noe fitting the model 
history = model.fit(img_train, label_train, epochs = 50, validation_split = 0.2)
# finding the best epoch
val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch))+1


Epoch 1/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.6325 - accuracy: 0.7750 - val_loss: 0.4162 - val_accuracy: 0.8448
Epoch 2/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3817 - accuracy: 0.8619 - val_loss: 0.3903 - val_accuracy: 0.8581
Epoch 3/50
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3384 - accuracy: 0.8745 - val_loss: 0.3811 - val_accuracy: 0.8622
Epoch 4/50
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3055 - accuracy: 0.8880 - val_loss: 0.3547 - val_accuracy: 0.8738
Epoch 5/50
1500/1500 [==============================] - 4s 3ms/step - loss: 0.2862 - accuracy: 0.8923 - val_loss: 0.3671 - val_accuracy: 0.8693
Epoch 6/50
1500/1500 [==============================] - 4s 3ms/step - loss: 0.2687 - accuracy: 0.9000 - val_loss: 0.3138 - val_accuracy: 0.8876
Epoch 7/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.2541 - accuracy: 0.9040 - val_loss: 0.3178 - val_accuracy:

## Retraining the model with best epoch found in the previous step 

In [19]:
hypermodel = tuner.hypermodel.build(best_hps)

#retraining with hte best epoch number
hypermodel.fit(img_train, label_train, epochs = best_epoch, validation_split=0.2)




Epoch 1/32
1500/1500 [==============================] - 5s 3ms/step - loss: 0.6301 - accuracy: 0.7769 - val_loss: 0.4015 - val_accuracy: 0.8562
Epoch 2/32
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3750 - accuracy: 0.8654 - val_loss: 0.3635 - val_accuracy: 0.8679
Epoch 3/32
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3287 - accuracy: 0.8797 - val_loss: 0.3410 - val_accuracy: 0.8760
Epoch 4/32
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3112 - accuracy: 0.8858 - val_loss: 0.3448 - val_accuracy: 0.8791
Epoch 5/32
1500/1500 [==============================] - 4s 3ms/step - loss: 0.2787 - accuracy: 0.8961 - val_loss: 0.3498 - val_accuracy: 0.8748
Epoch 6/32
1500/1500 [==============================] - 4s 3ms/step - loss: 0.2727 - accuracy: 0.9003 - val_loss: 0.3385 - val_accuracy: 0.8771
Epoch 7/32
1500/1500 [==============================] - 4s 3ms/step - loss: 0.2604 - accuracy: 0.9040 - val_loss: 0.3126 - val_accuracy:

## Evaluating the hypermodel on the test data 

In [20]:
eval_result = hypermodel.evaluate(img_test, label_test)
print("[test loss, test accuracy]:", eval_result)

313/313 [==============================] - 1s 2ms/step - loss: 0.4708 - accuracy: 0.8876
[test loss, test accuracy]: [0.47079238295555115, 0.8876000046730042]


# A slight overfit